In [1]:
%reload_ext autoreload
%autoreload 2

In [33]:
import pandas as pd
from tqdm import tqdm
from IPython.core.display import display, HTML
from gsmls import get_listing_detail_preview

def preview(mlsid):
    display(HTML(get_listing_detail_preview(mlsid)))

**Download listings from [GSMLS](https://www.gsmls.com/)**

In [46]:
from gsmls import get_listings

listings_list = []
for county in tqdm(['Morris']):
    listings_list += get_listings(county,
                           min_list_price=300000,
                           max_list_price=450000,
                           min_bedrooms=4,
                           min_bathrooms=2)

listings = {int(x['id']):x for x in listings_list}
    
print(f"Downloaded {len(listings)} listings from GSMLS.")

100%|██████████| 1/1 [00:06<00:00,  6.52s/it]

Downloaded 177 listings from GSMLS.


**Calculate commute time to NYC**

In [47]:
from geolocate import get_directions

destination = 'Port Authority Bus Terminal'
for id_, listing in tqdm(listings.items()):
    if not listing['address']: continue
    source = listing['address'] + ' ' + listing['city/town'] + ', ' + 'NJ'
    directions = get_directions(source, destination)
    listing['nyc_duration'] = directions['duration']['value'] if directions else None
    listing['nyc_duration_text'] = directions['duration']['text'] if directions else None
    listing['nyc_instructions'] = directions['instructions'] if directions else None

100%|██████████| 177/177 [02:26<00:00,  1.21it/s]


**Filter out listings**

In [48]:
# blacklist_styles = ['Ranch']
blacklist_cities = ['Paterson City', 'West Orange Twp.', 'Belleville Twp.', 'Elizabeth City']
df = pd.DataFrame(listings).T
df.drop('id', axis=1, inplace=True)
a = df[
    (df['address'].notnull()) &
    (df['style'] == 'Colonial') &
    (~df['city/town'].isin(blacklist_cities)) &
    (df['tax'] < 12000)]
len(a)

67

In [59]:
a.sort_values(by=['nyc_duration_text'], ascending=True).sort_values(by='sq_ft', ascending=False).head(10)

,address,bedrooms,city/town,county,full_baths,half_baths,nyc_duration,nyc_duration_text,nyc_instructions,price,rooms,sq_ft,style,tax,total_baths
3427375,6 Selmar Ter,5,East Hanover Twp.,Morris,3,1,5656,1 hour 34 mins,"[{'duration': '2 mins', 'distance': '463 ft', ...",399000,11,3300,Colonial,10477,3.1
3430526,178 Flanders Drakestown Rd,4,Mount Olive Twp.,Morris,2,1,9289,2 hours 35 mins,"[{'duration': '36 mins', 'distance': '1.8 mi',...",425000,9,3080,Colonial,11702,2.1
3407729,17 Highland Ave,4,Netcong Boro,Morris,3,0,5485,1 hour 31 mins,"[{'duration': '13 mins', 'distance': '0.7 mi',...",395000,9,2800,Colonial,9262,3
3412550,6 Jessica Ct,4,Jefferson Twp.,Morris,2,1,None,None,None,409000,10,2756,Colonial,11463,2.1
3427558,12 Pershing Ave,4,Mount Olive Twp.,Morris,2,1,8452,2 hours 21 mins,"[{'duration': '23 mins', 'distance': '1.2 mi',...",409900,9,2640,Colonial,11253,2.1
3430536,118 Pine St,5,Wharton Boro,Morris,4,0,6604,1 hour 50 mins,"[{'duration': '8 mins', 'distance': '0.4 mi', ...",369900,10,2500,Colonial,9132,4
3424615,14 Hopkins Ct,4,Mount Olive Twp.,Morris,2,1,None,None,None,364900,8,2437,Colonial,10916,2.1
3429623,3 N Glen Cir,4,Jefferson Twp.,Morris,2,1,None,None,None,425500,8,2428,Colonial,10497,2.1
3412166,1 Hickory Pl,5,Roxbury Twp.,Morris,2,1,None,None,None,334900,10,2401,Colonial,11217,2.1
3408431,18 Lisa Dr,4,Mount Olive Twp.,Morris,2,1,None,None,None,423000,8,2400,Colonial,11343,2.1


In [61]:
preview(3407729)